In [ ]:
#loads the language model, could take a while and requires ~60 GB of memory
lmDir = baseDir+'/languageModel'
ngramDecoder = lmDecoderUtils.build_lm_decoder(
    lmDir,
    acoustic_scale=0.8, #1.2
    nbest=1,
    beam=18
)

I0825 10:45:02.989113 64566 brain_speech_decoder.h:52] Reading fst /oak/stanford/groups/henderj/fwillett/speechPaperRelease_08_20/languageModel/TLG.fst


In [2]:
#to use the language model, make sure you've unzipped the languageModel.tar.gz file
#and have compiled the code in the LanguageModelDecoder folder
baseDir = '/home/nlp2023/data'

In [3]:
import os
from glob import glob
from pathlib import Path
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
from omegaconf import OmegaConf
import tensorflow as tf
import neuralDecoder.utils.lmDecoderUtils as lmDecoderUtils

In [70]:
#evaluate the RNN on the test partition and competitionHoldOut partition
from neuralDecoder.neuralSequenceDecoder import NeuralSequenceDecoder

testDirs = ['test','competitionHoldOut']
trueTranscriptions = [[],[]]
decodedTranscriptions = [[],[]]
for dirIdx in range(2):
    ckptDir = baseDir + '/derived/lstm-bi/baselineRelease'

    args = OmegaConf.load(os.path.join(ckptDir, 'args.yaml'))
    args['loadDir'] = ckptDir
    args['mode'] = 'infer'
    args['loadCheckpointIdx'] = None

    for x in range(len(args['dataset']['datasetProbabilityVal'])):
        args['dataset']['datasetProbabilityVal'][x] = 0.0

    for sessIdx in range(4,19):
        args['dataset']['datasetProbabilityVal'][sessIdx] = 1.0
        args['dataset']['dataDir'][sessIdx] = baseDir+'/derived/tfRecords'
    args['testDir'] = testDirs[dirIdx]

    # Initialize model
    tf.compat.v1.reset_default_graph()
    nsd = NeuralSequenceDecoder(args)

    # Inference
    out = nsd.inference()
    # print("###########################################################################################################")
    # print(dirIdx)
    # print(out)
    # decoder_out = lmDecoderUtils.cer_with_lm_decoder(ngramDecoder, out, outputType='speech_sil', blankPenalty=np.log(2))
    # break

    def _ascii_to_text(text):
        endIdx = np.argwhere(text==0)
        return ''.join([chr(char) for char in text[0:endIdx[0,0]]])

    for x in range(out['transcriptions'].shape[0]):
        trueTranscriptions[dirIdx].append(_ascii_to_text(out['transcriptions'][x,:]))  
    # decodedTranscriptions[dirIdx] = decoder_out['decoded_transcripts']
    break


Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.04.28/train
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.04.28/test
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.05/train
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.05/test
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.17/train
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.17/test
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.19/train
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.19/test
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.24/train
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.24/test
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.26/train
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.05.26/test
Load data from /home/nlp2023/data/derived/tfRecords/t12.2022.06.02/train
Load data from /home/nlp2023/data/derived/tfRecords/t12.2

In [71]:
print(out['logits'])
print(out['logits'].shape)

print(out['logitLengths'])
print(out['logitLengths'].shape)

print(out['decodedSeqs'])
print(out['decodedSeqs'].shape)

print(out['editDistances'])
print(out['editDistances'].shape)

print(out['trueSeqLengths'])
print(out['trueSeqLengths'].shape)

print("////////////////////////////////////////////////////////")

print(out['trueSeqs'])
print(out['trueSeqs'].shape)


print(out['transcriptions'])
print(out['transcriptions'].shape)

print(out['seqErrorRate'])
print(out['cer'])

[[[-0.00764113 -0.01309038 -0.09419972 ...  0.02092574 -0.06618738
    0.05294031]
  [ 0.00017801 -0.01906861 -0.09940473 ...  0.02825617 -0.09150063
    0.06027919]
  [ 0.00807436 -0.02114029 -0.10502458 ...  0.03670482 -0.11287102
    0.06846011]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.04098995 -0.02469903 -0.04661278 ...  0.01872456 -0.02660593
    0.04536364]
  [ 0.05406165 -0.0323121  -0.05844979 ...  0.02193451 -0.03716666
    0.05116723]
  [ 0.06558288 -0.03685975 -0.06985605 ...  0.02520989 -0.04422131
    0.05933619]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.01572515 -0.02752832 -0.054912

In [77]:
print(out['transcriptions'][0])
print(len(out['decodedSeqs'][0]))

[105 116  39 115  32 114 101  97 108 108 121  32 104  97 114 100  32 116
 111  32 102 105 110 100  32 115 111 109 101 116 104 105 110 103  32 116
 104  97 116  32 119 111 114 107 115   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [89]:
print(out['logits'][0][1])
print(max(out['logits'][0][1]))
print(out['logits'][0][1][32])

[ 0.00017801 -0.01906861 -0.09940473  0.0006286  -0.08326523  0.06384245
 -0.09280314  0.04570547  0.07534955 -0.07352786  0.0152181  -0.05364965
 -0.0846975  -0.01443802  0.0431516  -0.09346002  0.01303504  0.00078947
 -0.05759347 -0.03669511  0.02113678 -0.01306146 -0.05228222 -0.01917371
 -0.04488818  0.00566837  0.01335857  0.0111216   0.03325409  0.01734376
  0.08693416  0.0042457   0.10652    -0.03913116 -0.01579805  0.01684731
 -0.02506683  0.0357132   0.02825617 -0.09150063  0.06027919]
0.10652
0.10652


In [74]:
from neuralDecoder.utils.lmDecoderUtils import _cer_and_wer as cer_and_wer

#get word error rate and phoneme error rate for the test set (cer is actually phoneme error rate here)
cer, wer = cer_and_wer(decodedTranscriptions[0], trueTranscriptions[0], outputType='speech_sil', returnCI=True)

#print word error rate
print(cer)
print(wer)
print(trueTranscriptions[0][0])
print(chr(105))

/home/nlp2023/BCI/NeuralDecoder/neuralDecoder/utils/lmDecoderUtils.py:325: RuntimeWarning: invalid value encountered in scalar divide
  cer = np.sum(allCharErr) / np.sum(allChar)
/home/nlp2023/BCI/NeuralDecoder/neuralDecoder/utils/lmDecoderUtils.py:326: RuntimeWarning: invalid value encountered in scalar divide
  wer = np.sum(allWordErr) / np.sum(allWord)
/home/nlp2023/BCI/NeuralDecoder/neuralDecoder/utils/lmDecoderUtils.py:341: RuntimeWarning: invalid value encountered in scalar divide
  resampledCER[n] = np.sum(allCharErr[resampleIdx]) / np.sum(allChar[resampleIdx])
/home/nlp2023/BCI/NeuralDecoder/neuralDecoder/utils/lmDecoderUtils.py:342: RuntimeWarning: invalid value encountered in scalar divide
  resampledWER[n] = np.sum(allWordErr[resampleIdx]) / np.sum(allWord[resampleIdx])


(nan, nan, nan)
(nan, nan, nan)
it's really hard to find something that works
i


In [78]:
print(chr(32))

In [15]:
#print the sentence predictions for the test set
decodedTranscriptions = [[]]
decodedTranscriptions[0] = ["it's really hard to find something that works", 'and i love the line', 'i do have a friend that run', 'this way in the back', 'actually it makes sense to a certain extent', 'so the government that he was employed', "they feel good about what they're doing", "because it's an invasion of personal privacy", 'so you travel a lot', 'they were going to write', 'any kind of a good environment to live in', 'the best of android', "i don't know what you think about the point", 'i believe it cost about ten dollars', 'what in the world dogs are you on to', 'android tv box', 'a couple of times', 'i had the back down on it', 'which i you as a riff and have been real handy', 'i do when i was working', "it's a package deal", 'i have seen it', 'the measure always passed a senate committee', 'they told me that this was a topic', 'in the red square are weaver said', 'they did a lot of play work and stuff', 'you can gain too much right', 'recent legislation', "i'm a car buff too", 'two fans from a diesel heart', 'so in over trial never it all to suck', 'no cause cause air pollution', 'and then you people the case on top of that', 'just seeing that claire vision is a necessity', 'factors that affect earnings of an employee', 'if you drink a lot of caffeine will still work', "that's a nice car", 'visionary letters', "the only other place i've ever occasion", 'i just kind of went through', 'this is innovating voice we need', 'do you have your bag', 'so you know it keeps us pretty busy', "that's a sign in itself", 'so when you get in tight situations', 'i can relate to that', "it's all really quit playing", "if they're around anymore", 'i am not even aware that i could have seen it', 'the ranks of asian riders are feeling too', 'and turning discretion', 'most points at the end ones', "i think it's one hundred and eighty miles", "we don't usually take the kids when we go there", 'one of those generation models', "i'll never get caught", "i don't buy the newspaper", 'regions something for a potluck picnic', "i'll take it out", 'improved evasion does not work', 'casualties were reported', 'you just like to sit down and watch it', 'he helps by giving little pointers', 'cos they all grew up in new york city', 'you make such a choice', 'but they have snakes over there', 'a good positive traction', "that's what i was going to point out", 'i should run in about five hours early at noon', "it's a column nations type of company", 'i work out of my home', 'the way it works', "it doesn't have an air conditioner", 'then you have to make a salad', 'loving the bible in the equation', 'i read in the paper', "i'm in the peat processing business", 'once every year or two', "you don't eat it at all", "i've been tried and true", "i think that's something we might want", 'she thought for a few men', 'that is a sad situation', 'they do have capital punishment', 'it was nice talking to you dog', 'my favorite team is the rangers', 'kind of goes along with what you were saying', "why they don't go with foreign one", 'video choice award', 'they are the detroit dallas', 'he went to beat one man and came back another', 'as any city grow up', 'so you did buy a pen', "it's about taking and a half percent", 'i guess with the phone', "so i think it's going to be a lot easier", 'not too much soy sauce', 'we take a certain portion of your basic', 'who sent you', "and i don't know what's going to happen", "i think we have about thirteen months they're on it", 'i do personally find it on', 'they have clubs and a swimming pool', 'our point of view', 'it really is worth while to sew', 'i thought that was a little incentive', 'public transportation', 'focal point', 'i done one and one terabyte', 'what are we doing', 'my job and to be very demanding', 'you know it was just in last', 'and most people like that are old', "that's what i was doing before", 'the best teachers and the best students', "i don't like it cold", 'she was incurring in country', 'wait a minute', 'those are no good anymore', 'i do want to have a raised bad', 'i see that but that i know too', "it wasn't ugly enough for me", "if i don't know the jury line", "it's i've liked in the past", 'we came my treasure', 'he had choices to make a cancel call him', 'the last time he came was in march', "i'm getting pretty", "it's really a cost", 'have little mill calgary but won a lot more', 'you guys really are delayed', "we've got a son from austin", 'compression with depend', 'it still makes most of its source in asia', 'i think that is important too', 'have a good evening', 'and finally we can to make these occasion', "he didn't eat voice", "most of the time that don't bother with it", "they didn't see it so i got it back", "i'm sure all the eggs are in it", 'the family is that a disco', 'a lot of the area moves southward', "when disclosure isn't enough", 'contact points', 'this morning it was carrie dunn', 'satan rejoiced with his birth', 'they feel old because of the drug e', 'they are done with a fire turkey', 'note tech animation', 'that is reason for some people', "i'm away from my other son during those hours", 'where do you have to do a whole bunch of work', 'i take it she was an account', 'go to a tip for the incisions', 'the problem there at all', 'their way of life changed', 'past me that one back again came in', 'filter for a elation for hostels', 'i do that when we travel', 'i only saw the end of the first quarter', 'is there work to support this conclusion', 'by a training mission', 'as soon as we get their house minutes', 'and just keep chipping away at the us', 'all the money', 'my husband has a cool shed out back', 'what kind of sin', 'i always thought in my daughters with time out', 'know curry is still allowed to receive oil', "it's about getting parents more involved", 'this move that joint back account keeping', 'what you need is a tougher punishment regime', 'so it was really too late to do much', 'i want and to go to that golf course down south', 'you get a little taste of it and you say no', "they don't even take my turkey sultry summer", 'chicago and philadelphia', "i don't know if you have ever heard of it", 'social abra', "that's exactly the way he feels", 'do you really not remember', "it's am enjoyed that side of it", 'the ranks of asian writers are selling to', 'the message will tow the car and conclusions', 'burials usually happen during the morning', 'be kind to your whole self', "that's the other bar that we went to that night", "i don't know why", 'you can bake steaks love explosion penetrates her', "he's in first grade now", "i guess you some companies that aren't going fast enough", 'inherently dramatically warned', 'something a little extra', 'i kind of wish that they had a little lucky for the boys', 'killer bee in six', "i hadn't heard about that", 'the business of people to axis filter', "that's probably one of the reasons", 'quite a few movies are based off of that', 'but when i went home that all i could think about', 'i know of that actual', "a couple walked on here and 's", 'it was something as i could do', "i think they're a lot of people go to west virginia", 'been in black patches of fur', 'the jury is expected to announce its decision', 'some points', "i don't get to watch", 'all money now', "i'll help believes that more", 'and everything', 'they probably did it to their benefit', 'that is good that the like that', 'there is some different took in her report', 'i remain fickle times in killing bennett by on work', 'the new york cackle comes passed the measure', 'when you were in high school it was always fun', 'she knows how to do that', 'are you going to go see the exclusion of love', "i'm sure there are some", "that's kind of like what we'd like to do", 'quite a lot of boise', "i haven't been to a baseball game here", 'french can paint any more', 'a lesson of all and minna', 'it just kind of kid on forever', "that's the only way you can get to and from here", 'he taught a career low', 'we have the task excerpt from the draft', 'i have to think about it some more now', 'i was really kind of decide to start reading', 'what are you saying', 'your interests', 'i enjoyed it very much', 'he had occasion when her mother money', 'and earlier versions of this article', 'you might end be able to get a job there', 'enjoyable to you', 'we came to a park in the woods at night', "he will like it and i don't", 'it just seem small', 'i think it is confusion', 'just go compass for the day', "but they're going to learn it sometimes", 'it was the big one', "it's kind of a depression prop", 'every take months i have to pay my taxes', "i've never been chilling", 'i have six years of college', 'and then the next one down is in the marines', 'enjoy the adventure', 'not lately though', 'too many city problems for the long term', 'or the any other genre or sound or entrusted in explaining', "this i can't miss what they call a full summary", 'for family arena', "do you know that i've never seen a mac", 'i do both people', 'he recommends the hunters to a lot of people', 'do you have a commuter for yourself at home', "it's the original", "you can't blame them", 'more more more cannibals', 'i guess half could be an audience', "we've got four remaining food and lebanese food", 'i had to just and kin', 'football is an excellent point', "i'm glad it's on", "i don't know the details", 'then you pick more about the system of treasury decentralization', 'too many her story out there', 'if man city as a threat', 'go to the two and see elephants and dogs and things like that', "i'd be kind of fun", 'that appeared to be eight or nine years old', "i'm kind of out of it right now", "i don't write would every in the paper", 'others can do it', 'not everybody showed up', 'many efforts and five thirty', 'lawmakers passed a measure last year', 'i mean a large plate plate', "i'd like to see it be more of an import part", 'all of our tin cans', 'praying more often than usual', 'apple promised', "everyone knows they're going to get it", 'my in laws through to the kind of music i had', 'and on your computer or television', 'and we pay out here in the summertime', 'it never does one write again', 'you are personally applied', 'especially the ones that came out and said the piece', 'please on cost to you know', 'where is on point', 'palmer because', "it looks like they've got a women", "but you don't know that", 'i notice something it last time', "they won't have to", 'do you just like home cooking', 'so it seems to be', 'the decision to set the article', 'this wealth for employment cases', 'it would just depend', 'i like this season', "they've got the full pit room", 'you had to have a place', "that's need that you like cross settings also", 'the measure i had to come in', 'my three year old', 'just not feasible you know', "i can't really think of anything else offen", "i'm not sure how to go about it", "it's very tedious if you have a whole group", 'it was a cordless phone', "but it's kind of hard", 'you can do much with them', 'my still get out emissions by the sword', 'it was like four hundred times as much', "it's possibly done", 'i would feel that i had the right to', 'you just call him react', 'i know the is', 'it really hurt mr', 'the experience', 'do we have to wait until all the line fail', 'really scary to me', 'i think it is getting worse', 'i really would like to see them too well', "i don't live in a house", 'people put them in', 'it had come away fast enough and go hide', 'for about two hundred dollars', 'in that time frame', 'trying to make it through that', 'anything like that we provide', 'seven years in the polls', 'many years or so', 'i think it does help', 'that has got to be common this time of their', 'even this pasta', 'can you talk live', "it's been kind of cookie", 'the games are just read', 'we try to go so offen', 'it took four days', 'the pros and cons of both', "we're so bad", 'you get program', 'not that i like writing', 'i think a cup or two', "i don't mean to be making", "i'd like to see those", 'a controlled something', 'real close', 'i guess you find on this', 'it might be kind of tech', 'perhaps not in the matter that we do it', 'do you have found for older children', 'one had more pull to it', 'i just feel valuable in time that every year', 'i get up on a farm', "that was the one that's been really nice to have", "i've done it before", 'the end was pretty good', "how different the people they're away", 'we get out on the water occasionally', 'maybe once every call or two', "it's been with fun", 'i bet that once in a while', 'how to do things', 'i happily im not a cat person', 'the lake mayor that you get', 'they put him in the naughty office', 'since this time last year of change', "they didn't fight together", 'i have to know what he was doing', 'they just live in the community', 'it had to be a member anyway', 'if you do decide to go with child care', 'the insurance pay for the body shaming', 'one is eight and one is eleven', 'i was drug tested', 'new names and developing companies', 'they are going to get a gun no matter what', 'when i was in high school we had a choice', 'ever printed the it', "that's how we got started", 'a nice guess than i could avoid', "it's really real", 'more people want', 'i think what they will do is start paying mexico', "i haven't expect it", 'you have to have a pretty good team', 'their small little rocks and cars like that', "they don't have the ability to develop it", 'we got to have i want them under', "there's so much going on here", "the victims' families and things", 'playing one of these sit in a mother', 'the drug dealers', 'my wife mother is the only gained in life', "they're good now", 'i have never seen before', "i don't know what we're going to do", "it's second nature", 'find out where the languages are', "we don't get to see too many", "it's like a bolt going by", 'i mean like today was eighteen dollars', "it's really unfortunate", 'the next morning you want to take them out', 'if the new we can is any good', 'if something goes wrong', "i haven't seen any", "this to say it's either all through the house", "you don't worry about them too much", 'that kind of gas mileage', 'it really does make any difference', 'i was one in texas and a red other years', 'i guess because i work get a bit with a', "i guess i haven't had that much experience", 'reports and paper and that sort of thing', 'planning means that does not belong to them', 'does it have to be a special color', "that's the freedom of choice", "i don't know too much about it", 'my son play football', 'see how the family like it', 'one in college away', 'his civil rights violated', 'it looks like it would be', 'going under by hand and that kind of thing', "i don't want top in my car", 'we can keep that out of our minds', 'from for his party banners', 'we are seeing that here as well', "that's what made me happy", 'it depends on the person', "i don't make that much", "it's been a long time", 'no one can live on ice', 'maybe they passed it on that measure arm move', 'it seems like they would be able to', "the didn't point the car", "she's one of those that love", 'so i just have to face it though', 'mostly with adolescents', "i'm not a company or anything like that", 'i like to listen to reno music a whole lot', 'to our classes', 'basically allow', 'do you enjoy camping', "it's kind of a show pieces", "amazon's down there", 'how do you like your should be', 'it was just two people at all times', 'their self edit is wrong', 'it is pretty okay', 'i thought that that was what sally did was for', "i couldn't get her up", 'with it six weeks old', 'fifteen feet wide', 'you a bigger city', 'wes lunt and tape it in the fund', 'when you find out who it was', 'because the big guns to school', "they're well up", 'have you ever been drug tested', 'he could get hurt', "i'm originally a second", "so you've got a lot of traffic knowns real close", 'really doing it quite a bit', 'i know a girl', "we've got to eat most of the time", 'to the place of what a b', 'even in my case', 'can do whatever they want', 'i had the part', 'we see around and it popping', "i don't have boys", 'that would have been missed time to go', "if that doesn't have to", "that's certainly fair", 'you develop for attention', 'we have that here', 'usually pays fine for survival', "i've had so many people", "i don't even get to see have my project", 'we put some papers on the floor', "that's the closest thing", 'that kind of funny actually', 'it was was the back i guess', 'pop up here', 'you with the home office now', 'they seem to be a comically given all', 'that be a really good thing to do', 'too much business', "he's retired now", 'i heard this on a christian program', "even though they're too close", 'and his money', 'i had never seen anything', "this week's people started", 'now to be takes back', 'i caught a cop', 'i feel so silly', 'are they all here in the area', 'i kind of look at it this way', 'public auction', 'sewing and quilting', 'good to hear from', 'so aka c', 'i read a lot and then i know people', 'did you pull them', 'that would make me happy though', "it's nice to get out in the open air", 'i think it would do something for it', 'it has so been nigh here', 'seats that went college', 'it worked out real news', "we've had lot of them in the past", "it's not been that way in the last five years", 'i live out in the middle of it', "it's depressing too", "i don't own my home", "i've always wanta how you get around", "it's just tough", 'raking muck that i really like', 'you got any more', "i'm going to think it's not", "that's something i missed quite a bit", 'they finally came up with evidence', "that's where i didn't think about that", 'good data as a team', 'but of it would probably work out', "it's necessarily going to involve more taxes", "it's just going to follow", "i'm kind of a car buff myself", 'buying of our face', 'sometimes it will lapse', "i just don't like that", 'social training to talk about', 'i am close to retrieve my english degree', 'everything to enjoy', 'i made a real big take himself', "there's actual right for them", 'it was for a murder trial', 'are you a professional person', "it's far beyond me", "we've been an awful lot from the space program", 'two sets of conference', 'i call and find out', 'you want sally to bust right', "i've heard about him", "we've had more freezing this winter", "if we can't get a handle on that", 'they would vote for choice', 'they were ok', 'to find a button', 'on a working time', 'after we had them for several years', 'if they work it it a little bit', "we don't have enough", 'what are you making', 'absolutely nothing', 'i had a carpet service call up the time', "although he's probably been in a lot of them", 'just about any kind of music', 'we like mason wood', 'i know the day care center are not cheap either', 'i found it a lot of time', "they've got to find and everyone gone", 'i think he really makes it', 'where you got the pie', 'it all start to become feral', 'i could go along with that', "it's like one taken of accordion", "that's a big thing with me", "i'm the only one", "i've been following through on that", 'if there start to be a lot of violence l', 'walking seems to be the best', "i don't think you made of them are anymore", 'how do you feel about your game', 'when i leave that in the country', "i'm originally from maine", 'you grow up and to decide what they want to do', 'we just get sick of it on before', 'i had friends', "i've worked in all", 'they can pretty much got everybody there', "you can kiss c i'm sorry", 'the engine build up in it', 'the arm pushes it too much', "i don't have a lot of time", 'i know this well', "sometimes they're not very hard"]

print(decodedTranscriptions[0])

["it's really hard to find something that works", 'and i love the line', 'i do have a friend that run', 'this way in the back', 'actually it makes sense to a certain extent', 'so the government that he was employed', "they feel good about what they're doing", "because it's an invasion of personal privacy", 'so you travel a lot', 'they were going to write', 'any kind of a good environment to live in', 'the best of android', "i don't know what you think about the point", 'i believe it cost about ten dollars', 'what in the world dogs are you on to', 'android tv box', 'a couple of times', 'i had the back down on it', 'which i you as a riff and have been real handy', 'i do when i was working', "it's a package deal", 'i have seen it', 'the measure always passed a senate committee', 'they told me that this was a topic', 'in the red square are weaver said', 'they did a lot of play work and stuff', 'you can gain too much right', 'recent legislation', "i'm a car buff too", 'two fans from a diese

In [7]:
#print the predictions for the competition hold-out set (labels are unreleased)
print(decodedTranscriptions[1])

["i'm initially from colorado", "if i had money i'd be safe to buy a known", 'i raced in york', 'i thought the topic was pouring', 'what every drive up there', "you don't get evolve", 'i hope you enjoy', "we don't do a", 'a voice from the crowd', 'i hope you enjoy my blog', 'i never have it', 'maybe even too far', "it's had that russians were quite some time", 'quick animus or suit or dress', 'manufactured by nineteen seventy five', 'the water takes the recession', "because that's all she talked about", 'did you hear an piece about that', 'i play in several company sources', "i've heard on the news", 'maybe but in a little bit fit', 'and i guess it was late me', 'in just cause', 'the word of the confusion', 'and it evoked an emotional response', 'you want to buy something', "that's basically what we do", 'i guess a lot of people do', 'a lot of people have real water', 'it seems like he would develop pride', "but you're not", 'they make a collection on monday for yard waste', 'several o

In [8]:
#format the predictions for competition submission. This generates a .txt file that can be submitted.
with open('baselineCompetitionSubmission.txt', 'w') as f:
    for x in range(len(decodedTranscriptions[1])):
        f.write(decodedTranscriptions[1][x]+'\n')